In [ ]:
from bisect import bisect_left, bisect_right
import math
import os
import sys

import numpy as np
import pandas as pd
from scipy.interpolate import griddata
import plotly as plt
import plotly.graph_objects as go

In [ ]:
sys.path.append(os.path.expanduser("../src"))
from spinorama.load_klippel import parse_graph_freq_klippel

from spinorama.compute_scores import octave
from spinorama.plot import contour_params_default, plot_contour, CONTOUR_COLORSCALE
from spinorama.compute_misc import compute_contour
from spinorama.load import filter_graphs

In [ ]:
speaker_name = "Genelec 8361A"
speaker_origin = "asr-vertical"
speaker_orientation = "Horizontal"
status, (name, spl_h) = parse_graph_freq_klippel(
    "../datas/measurements/{}/{}/SPL Horizontal.txt".format(speaker_name, speaker_origin)
)
status, (name, spl_v) = parse_graph_freq_klippel(
    "../datas/measurements/{}/{}/SPL Vertical.txt".format(speaker_name, speaker_origin)
)
df = filter_graphs(speaker_name, spl_h, spl_v)
# df.keys()

In [ ]:
freq, angle, spl = compute_contour(df["SPL {}_normalized_unmelted".format(speaker_orientation)])
print(freq.shape, angle.shape, spl.shape)

In [ ]:
fig = plot_contour(
    df["SPL {}_normalized_unmelted".format(speaker_orientation)], contour_params_default
)
fig.update_layout(width=500, height=400).show()

In [ ]:
def remap(freqs, angles, spls, xlen):
    # make first freq 0 in the log graph
    # scaled_freqs = freqs*20000/np.max(freqs)
    # scaled_freqs /= np.min(scaled_freqs)
    scaled_freqs = freqs / np.min(freqs)
    min_f = np.log10(np.min(scaled_freqs))
    max_f = np.log10(np.max(scaled_freqs))
    log_freqs = np.log10(scaled_freqs)
    # polar view
    x = np.multiply(log_freqs, np.cos(np.deg2rad(angles + 90))).flatten()
    y = np.multiply(log_freqs, np.sin(np.deg2rad(angles + 90))).flatten()
    z = spls.flatten()
    print(
        "freq range {}Hz -- {}Hz ; log range {}--{} ; n={}".format(
            np.min(freqs), np.max(freqs), min_f, max_f, len(freqs[0])
        )
    )
    # grid
    map_x = np.linspace(-max_f, max_f, xlen)
    map_g = np.meshgrid(map_x, map_x)
    map_v = [(i, j) for i in map_x for j in map_x]
    # remap on the new grid
    map_z = griddata((x, y), z, map_v, method="cubic")
    map_z = 3 * (map_z // 3)
    map_z = [
        (
            map_z[i * xlen + j]
            if xi**2 + xj**2 <= max_f**2 and xi**2 + xj**2 >= min_f**2
            else math.nan
        )
        for i, xi in enumerate(map_x)
        for j, xj in enumerate(map_x)
    ]
    return map_g, np.clip(map_z, -30, 3), (np.min(freqs), np.max(freqs))


def slice2contour(freq, angle, spl, a, b, xlen):
    # extraxct slices
    sliced_freq = freq.T[a:b].T
    sliced_angle = angle.T[a:b].T
    sliced_spl = spl.T[a:b].T
    return remap(sliced_freq, sliced_angle, sliced_spl, xlen)


freq, angle, spl = compute_contour(df["SPL Horizontal_normalized_unmelted"])
data_g, data_spl, data_range = slice2contour(freq, angle, spl, 0, len(freq[0]), 150)
fig = go.Figure(
    data=[
        # go.Scatter(
        #    x=data_g[0].flatten(),
        #    y=data_g[1].flatten(),
        #    marker_color=3*data_spl//3,
        #    mode='markers'
        # ),
        go.Contour(
            x=data_g[0].flatten(),
            y=data_g[1].flatten(),
            z=data_spl,
            contours=dict(start=-30, end=3, size=3),
            colorscale=CONTOUR_COLORSCALE,
            line_width=0,
            connectgaps=False,
            line_smoothing=0.85,
            contours_coloring="heatmap",
        ),
    ],
    layout=go.Layout(
        title=None, width=400, height=400, template="none", xaxis_visible=False, yaxis_visible=False
    ),
)
fig.show()

In [ ]:
def measurements2polars(freq, angle, spl, xlen):
    octave_min = 100
    octave_max = 20000
    octave_bands = 2
    valid_bands = []
    nb_freqs = len(freq[0])
    datas = []
    # for fmin, fcenter, fmax in octave(octave_bands):
    #    if fmax<octave_min or fmin>octave_max:
    #        continue
    #    a = bisect_right(freq[0], fmin)
    #    b = min(nb_freqs-1, bisect_left(freq[0], fmax))
    #    #if b-a<10:
    #    #    break
    #    valid_bands.append((fmin, fcenter, fmax))
    #    data = slice2contour(freq, angle, spl, a, b, xlen)
    #    datas.append(data)
    steps = 10
    mod = len(freq[0]) % steps - 1
    for i in range(0, len(freq[0]) // steps):
        a = i * steps + mod
        if a < 0:
            continue
        b = (i + 1) * steps + mod
        fmin = freq[0][a]
        fmax = freq[0][b]
        fcenter = freq[0][(a + b) // 2]
        if fmin < 200:
            continue
        valid_bands.append((fmin, fcenter, fmax))
        data = slice2contour(freq, angle, spl, a, b, xlen)
        datas.append(data)
    return datas, valid_bands

In [ ]:
def datas2menu():
    return [
        {
            "buttons": [
                {
                    "args": [
                        None,
                        {
                            "frame": {"duration": 500, "redraw": True},
                            "fromcurrent": True,
                            "transition": {"duration": 300, "easing": "quadratic-in-out"},
                        },
                    ],
                    "label": "Play",
                    "method": "animate",
                },
                {
                    "args": [
                        [None],
                        {
                            "frame": {"duration": 0, "redraw": True},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": "Pause",
                    "method": "animate",
                },
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 80},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top",
        }
    ]


def datas2sliders(valid_bands):
    return {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Freq:",
            "visible": True,
            "xanchor": "right",
        },
        "transition": {"duration": 150, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": [
            {
                "args": [
                    [str(i)],
                    {
                        "frame": {"duration": 500, "redraw": True},
                        "mode": "immediate",
                        "transition": {"duration": 300},
                    },
                ],
                "label": "{}Hz".format(int(fcenter)),
                "method": "animate",
            }
            for i, (fmin, fcenter, fmax) in enumerate(valid_bands)
        ],
    }


def datas2frames(datas, name, orientation):
    return [
        (
            go.Contour(
                x=current_g[0].flatten(),
                y=current_g[1].flatten(),
                z=current_map_spl,
                contours=dict(start=-30, end=3, size=3),
                colorscale=CONTOUR_COLORSCALE,
                line_width=0,
                connectgaps=False,
                line_smoothing=0.85,
                contours_coloring="heatmap",
            ),
            go.Layout(
                title="{} {} {}--{}Hz".format(
                    name, orientation, current_freq_range[0], current_freq_range[1]
                ),
                width=600,
                height=680,
                template="none",
                xaxis_visible=False,
                yaxis_visible=False,
            ),
        )
        for current_g, current_map_spl, current_freq_range in datas
    ]


def polars2plot(datas, valid_bands, name, orientation):
    update_menus = datas2menu()
    sliders = datas2sliders(valid_bands)
    frames = datas2frames(datas, name, orientation)
    frame0 = frames[0]
    data0 = frame0[0]
    layout0 = frame0[1]
    layout0["updatemenus"] = update_menus
    layout0["sliders"] = [sliders]
    go_frames = [
        go.Frame(data=contour, layout=layout, name=str(i))
        for i, (contour, layout) in enumerate(frames)
    ]
    fig = go.Figure(data=data0, layout=layout0, frames=go_frames[1:])
    return fig, frames

In [ ]:
def speaker2animation(name, orientation, normalized, df, xlen):
    key = "SPL {}_unmelted".format(orientation)
    if normalized:
        key = "SPL {}_normalized_unmelted".format(orientation)
    freq, angle, spl = compute_contour(df[key])
    datas, valid_bands = measurements2polars(freq, angle, spl, xlen)
    return polars2plot(datas, valid_bands, name, orientation)

In [ ]:
fig, frames = speaker2animation(speaker_name, "Horizontal", True, df, 250)
fig.show()

In [ ]:
import io
import cv2
from PIL import Image


def fig2array(fig):
    # convert Plotly fig to  an array
    fig_bytes = fig.to_image(format="png")
    buf = io.BytesIO(fig_bytes)
    img = Image.open(buf)
    return np.asarray(img)


def buildVideo(videoname, images, size):
    out = cv2.VideoWriter(videoname, cv2.VideoWriter_fourcc(*"MPEG"), 3, size)
    for image in images:
        out.write(image)
    out.release()


images = [fig2array(go.Figure(data=contour, layout=layout)) for contour, layout in frames]

In [ ]:
buildVideo("{}-{}.mp4".format(speaker_name, speaker_orientation), images, (675, 600))

In [ ]:
for i, (contour, layout) in enumerate(frames):
    fig = go.Figure(data=contour, layout=layout)
    fig.write_image("/tmp/{}-{}.png".format(speaker_name, i))

In [ ]:
images = [cv2.imread("/tmp/{}-{}.png".format(speaker_name, i)) for i in range(len(frames))]

In [ ]:
buildVideo("{}-{}.mp4".format(speaker_name, speaker_orientation), images, (750, 600))